<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/0530/0530_04_Word2Vec_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

파이썬의 gensim 패키지에는 Word2Vec을 지원하고 있어, gensim 패키지를 이용하면 손쉽게 단어를 임베딩 벡터로 변환시킬 수 있습니다. 영어로 된 코퍼스를 다운받아 전처리를 수행하고, 전처리한 데이터를 바탕으로 Word2Vec 작업을 진행하겠습니다

Word2Vec()의 파라메터
- vector_size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
- window = 컨텍스트 윈도우 크기
- min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
- workers = 학습을 위한 프로세스 수
- sg = 0은 CBOW, 1은 Skip-gram.

## 실습 : 한국어 Word2Vec 실습

In [ ]:
!pip install konlpy

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from tqdm import tqdm
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt

네이버 영화 리뷰 데이터로 한국어 Word2Vec을 만들어봅시다.데이터를 다운로드 합니다.

In [ ]:
train = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', header=0, delimiter='\t' ,quoting=3)
test = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', header=0, delimiter='\t' ,quoting=3)

In [ ]:
train.shape, test.shape

((150000, 3), (50000, 3))

In [ ]:
train

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [ ]:
test

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


### 데이터 탐색 및 전처리

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [ ]:
# 결측치 처리 
train = train.dropna(how = 'any') # Null 값이 존재하는 행 제거
test = test.dropna(how = 'any') # Null 값이 존재하는 행 제거
print('NULL 값 존재 유무 :', train.isnull().values.any()) # Null 값이 존재하는지 확인
print('NULL 값 존재 유무 :', test.isnull().values.any()) # Null 값이 존재하는지 확인

NULL 값 존재 유무 : False
NULL 값 존재 유무 : False


In [ ]:
import re
re.sub

In [ ]:
# 정규 표현식을 통한 한글 외 문자 제거
train['document'] = train['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True) 
test['document'] = test['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True)
# import re와 str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True) 해도 동일한 결과나옴
# str.replace( ,regex=True)하면 정규표현식 쓸 수 있음

<ipython-input-9-84d25046e4e1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['document'] = train['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True)
<ipython-input-9-84d25046e4e1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['document'] = test['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True)


In [ ]:
test[:5] 

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0


In [ ]:
train[:5] 

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [ ]:
# okt = Okt()
# okt.morphs :  형태소 분석

In [ ]:
# 형태소 분석기 OKT를 사용한 토큰화 작업 - train
okt = Okt()

tokenized_train = []
for sentence in tqdm(train['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    tokenized_train.append(tokenized_sentence)

100%|██████████| 149995/149995 [05:35<00:00, 447.15it/s]


In [ ]:
# 형태소 분석기 OKT를 사용한 토큰화 작업 - test
okt = Okt()

tokenized_test = []
for sentence in tqdm(test['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    tokenized_test.append(tokenized_sentence)

100%|██████████| 49997/49997 [02:00<00:00, 413.75it/s]


In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = tokenized_train, vector_size = 30, window = 5, min_count = 5, workers = 4, sg = 0)
# workers : 멀티 코어로 하겠다
# sg = 0 : CBOW / 1 = SKIP GROW
# sg : Word2Vec 생성하는 모델 설정

In [ ]:
model.wv.vectors

array([[ 2.1930902e-01, -5.2315897e-01,  1.2969488e+00, ...,
        -1.8854566e-02, -1.9721727e+00, -1.7456466e+00],
       [-9.5246977e-01, -1.5169467e+00,  2.0100818e+00, ...,
         1.1390183e+00, -2.1992157e+00, -3.7441943e-02],
       [-2.1921200e-01,  9.2588305e-01, -1.1093744e+00, ...,
        -8.4637111e-01, -2.8082123e+00,  3.7935790e-01],
       ...,
       [ 4.8221485e-03, -4.1674506e-03,  2.0499364e-02, ...,
         3.7800666e-02,  4.5444924e-02, -2.1969758e-01],
       [-6.7804649e-05, -3.5619911e-02,  8.3624803e-02, ...,
         7.2797433e-02,  3.0753478e-02, -1.3091147e-01],
       [ 7.4633178e-03, -8.9239962e-02,  8.1253260e-02, ...,
         6.2389906e-02,  8.7305747e-02, -1.4699709e-01]], dtype=float32)

In [ ]:
print('완성된 임베딩 매트릭스의 크기 확인 :', model.wv.vectors.shape) 
# 단어의 모양, 문장의 모양이 아님 (분석하려면 문장 필요)
# 문장이라면 맨 처음 넣은 문장의 개수가 나와야 함(document = 149995)
# 동음이의어를 구분할 수 없음 = 중복값 없이 나옴

완성된 임베딩 매트릭스의 크기 확인 : (14272, 30)


In [ ]:
print(model.wv.index_to_key)
print(len(model.wv.index_to_key)) # 14272 개 나옴 = 단어, 

In [ ]:
model.wv.vectors[0] # 문장이 아니라 단어

In [ ]:
print(model.wv.most_similar("최민식"))

[('김수현', 0.9027039408683777), ('조재현', 0.8911566734313965), ('이민호', 0.8854854702949524), ('한석규', 0.8730553984642029), ('김명민', 0.8699944019317627), ('설경구', 0.8687490820884705), ('차승원', 0.8679012656211853), ('메릴', 0.866558313369751), ('전도연', 0.8664982914924622), ('스트립', 0.8601122498512268)]


In [ ]:
print(model.wv.most_similar("히어로"))

[('슬래셔', 0.8700234889984131), ('호러', 0.8605641722679138), ('뱀파이어', 0.8582934141159058), ('느와르', 0.8521326780319214), ('블록버스터', 0.8508649468421936), ('무협', 0.8429354429244995), ('좀비', 0.8334152698516846), ('홍콩', 0.826463520526886), ('무비', 0.8238034844398499), ('갱스터', 0.8199294805526733)]


In [ ]:
print(model.wv.most_similar("연기"))

[('연기력', 0.8502976894378662), ('목소리', 0.7702709436416626), ('발연기', 0.7477604746818542), ('케미', 0.7163610458374023), ('캐스팅', 0.6872141361236572), ('소화', 0.6711622476577759), ('외모', 0.6657480597496033), ('역할', 0.6497616767883301), ('씨', 0.6430972218513489), ('노래', 0.6427987217903137)]


In [ ]:
# 완성된 임베딩 매트릭스의 크기 확인
model.wv.vectors.shape
# 문장에 대한 것은 만들어주지 않기 때문에 사람이 문장에 대한 벡터를 만들어줘야 한다(2차원으로 만들어야 모델에 들어간다)

(14272, 30)

In [ ]:
def get_document_vectors(document_list):
    document_embedding_list = []

    # 각 문서에 대해서
    for line in document_list:
        doc2vec = None
        count = 0
        for word in line:
            if word in model.wv.key_to_index.keys():
                count += 1
                # 해당 문서에 있는 모든 단어들의 벡터값을 더한다.
                if doc2vec is None:
                    doc2vec = model.wv[word] #doc2vec에 있으면 있는 그대로 넣는다
                else:
                    doc2vec = doc2vec + model.wv[word] #doc2vec에 있으면 추가해서 넣는다

        if doc2vec is not None:
            # 단어 벡터를 모두 더한 벡터의 값을 단어 개수로 나눠준다.
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)
        
        else:
            #빈 리스트가 나오면 제로행렬로 채운다 (사람에 따라 빈 리스트는 없애는 코드를 사용하기도 한다)
            document_embedding_list.append(np.zeros(30))

    # 각 문서에 대한 문서 벡터 리스트를 리턴
    return np.array(document_embedding_list)

In [ ]:
X_train = get_document_vectors(tokenized_train)
print('문서 벡터의 수 :',len(X_train)) # 문서 벡터는 사람이 구성해야 한다

문서 벡터의 수 : 149995


In [ ]:
X_test = get_document_vectors(tokenized_test)
print('문서 벡터의 수 :',len(X_test))

문서 벡터의 수 : 49997


In [ ]:
y_train = train['label']
y_test = test['label']

y_train.shape, y_test.shape

((149995,), (49997,))

In [ ]:
# 모델 1 : Logistic Regression 모형
from sklearn.linear_model import LogisticRegression

log_clf = LogisticRegression()
log_clf.fit(X_train, y_train)
print('Train set score: {:.3f}'.format(log_clf.score(X_train, y_train)))
print('Test set score: {:.3f}'.format(log_clf.score(X_test, y_test)))

Train set score: 0.787
Test set score: 0.785


In [ ]:
# 모델 2 : 트리 앙상블 모형
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier()
model_rf.fit(X_train, y_train)
print('Train set score: {:.3f}'.format(model_rf.score(X_train, y_train)))
print('Test set score: {:.3f}'.format(model_rf.score(X_test, y_test)))

Train set score: 0.991
Test set score: 0.794


In [ ]:
# 모델 3 : lightgbm Model
import lightgbm as lgb

model_lgbm = lgb.LGBMClassifier()
model_lgbm.fit(X_train, y_train)
print('Train set score: {:.3f}'.format(model_lgbm.score(X_train, y_train)))
print('Test set score: {:.3f}'.format(model_lgbm.score(X_test, y_test)))

Train set score: 0.809
Test set score: 0.793
